In [20]:
!pip install langgraph langchain_anthropic

In [2]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")
_set_env("ANTHROPIC_API_KEY")

ANTHROPIC_API_KEY:  ········


In [3]:
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field
from typing import Optional

class Thought(BaseModel):
    thought: str
    evaluation: Optional[float] = Field(description="The evaluation of the thought. It can be a number between 0.1 and 1.0 being 0.1 the worst and 1.0 the best.")
    
model = ChatAnthropic(model="claude-3-5-sonnet-latest")

thinker = model.with_structured_output(Thought)
print(thinker.invoke("Hello How are you"))

thought="I'm an AI assistant, and I'm functioning well and ready to help. This is a polite greeting from the user that deserves a courteous response." evaluation=0.8


In [34]:
from langgraph.prebuilt import create_react_agent
import uuid
import json
from langchain_core.tools import tool
from typing import Any, Dict
from concurrent.futures import ThreadPoolExecutor

TOT_SYS_PROMPT = """
You are an expert problem-solving agent designed to not only solve complex problems but also critically evaluate the quality of your thought process and final answers. 
Your task is to follow a structured approach to generate solutions, assess your thoughts, and provide a rating for each on a scale of 0.1 to 1.0. 
This rating should reflect the accuracy and quality of your reasoning and final answer.

### Instructions:

1. **Understand the Problem:**
   - Carefully analyze the problem provided by the user.
   - Break down the problem into smaller, manageable parts if necessary.
   - Formulate a clear understanding of the problem before proceeding.

2. **Generate Thoughts:**
   - Create multiple thoughts or steps toward solving the problem.
   - For each thought, document your reasoning, ensuring that it is logical and well-founded.

3. **Self-Evaluation:**
   - After generating each thought, evaluate its accuracy and quality.
   - Assign an evaluation score between 0.1 and 1.0. Use the following guidelines:
     - **0.1 to 0.4:** The thought is flawed, inaccurate, or incomplete.
     - **0.5 to 0.7:** The thought is partially correct but may lack detail or full accuracy.
     - **0.8 to 1.0:** The thought is accurate, complete, and well-reasoned.

4. **Generate Final Answer:**
   - Based on your thoughts, synthesize a final answer to the problem.
   - Ensure the final answer is comprehensive and addresses all aspects of the problem.

5. **Final Evaluation:**
   - Evaluate the overall quality and accuracy of your final answer.
   - Provide a final evaluation score based on the same 0.1 to 1.0 scale.
   
"""


@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b

class ToTAgent:
    def __init__(
        self,
        model: any,
        threshold: float,
        max_loops: int,
        prune_threshold: float = 0.5,
        number_of_processes: int = 3,
        id: str = uuid.uuid4().hex,

    ):
        self.id = id
        self.model = model
        self.threshold = threshold
        self.max_loops = max_loops
        self.prune_threshold = prune_threshold
        self.all_thoughts = []  # Store all thoughts generated during DFS
        self.pruned_branches = []  # Store metadata on pruned branches
        self.number_of_processes = number_of_processes


        

    def dfs(self, state: str, step: int = 0)-> Optional[Dict[str,Any]]:
        print(f"\n/-----Starting dfs for state: {state}----/\n")

        if step>= self.max_loops:
            return None

        with ThreadPoolExecutor(max_workers=self.number_of_processes) as exec:
            next_thoughts = list(
                exec.map(self.model.invoke, [state] * self.number_of_processes)
            )

            next_thoughts.sort(key=lambda x: x.evaluation, reverse=False)

            for thought in next_thoughts:
                if thought.evaluation > self.prune_threshold:
                    self.all_thoughts.append(thought)
                    result = self.dfs(thought.thought, step + 1)

                    if result and result.evaluation > self.threshold:
                        return result
                    else:
                        self._prune_thought(thought)

        
    def _prune_thought(self, thought: Dict[str, Any]):
        self.pruned_branches.append(
                {
                "thought": thought.thought,
                "evaluation": thought.evaluation,
                "reason": "Evaluation score below threshold",
                }
             )

    def run(self, task:str) -> str:

            root_thoughts = self.dfs(task)

            for i in range(1, self.max_loops):
                if root_thoughts:
                    next_task = root_thoughts.thought
                    initial_thoughts = self.dfs(next_task, step=i)
                else:
                    break

            self.all_thoughts.sort(key=lambda x: x.evaluation, reverse=False)
            tree_of_thoughts = {
                "final_thoughts": [x.model_dump() for x in self.all_thoughts],
                "pruned_branches": self.pruned_branches,
                "highest_rated_thought": (
                    self.all_thoughts[-1].model_dump() if self.all_thoughts else None
                ),
            }

            output_str = json.dumps(tree_of_thoughts, indent = 4)
            return output_str
            

In [35]:
tot_agent = ToTAgent(
    model=thinker,
    threshold=0.8,
    max_loops=1,
    prune_threshold=0.5,
    number_of_processes=4
)

initial_state = """

Your task: is to use 4 numbers and basic arithmetic operations (+-*/) to obtain 24 in 1 equation, return only the math

"""


final_thought = tot_agent.run(initial_state)

print(final_thought)


/-----Starting dfs for state: 

Your task: is to use 4 numbers and basic arithmetic operations (+-*/) to obtain 24 in 1 equation, return only the math

----/


/-----Starting dfs for state: Let me solve this by working systematically to find a valid combination of 4 numbers and operations that equals 24.----/


/-----Starting dfs for state: Let me solve this using 4 numbers and basic arithmetic to get 24. A straightforward solution would be: 6 * 4 = 24, but we need to use 4 numbers. Let me think about a clear solution using 4 numbers.----/


/-----Starting dfs for state: Let me solve this step by step:
1) I'll use common numbers 3,4,5,2
2) To get 24, I can multiply 3 by 4 to get 12, then multiply by 2 to get 24
3) The equation would be: 3 * 4 * 2 + 0 * 5 = 24----/

{
    "final_thoughts": [
        {
            "thought": "Let me solve this by working systematically to find a valid combination of 4 numbers and operations that equals 24.",
            "evaluation": 0.8
        },
    